In [1]:
import os
import pandas as pd
from typing import List, Tuple, Dict
from collections import defaultdict

In [2]:
main_dir = '/Users/marjan/Desktop/project/'
UD_dir = os.path.join(main_dir, 'data', 'UD_Persian-PerDT', 'fa_perdt-ud-train.conllu')
persian_adj_dir = os.path.join(main_dir, 'output', 'Persian', 'persian_adjs.csv')
persian_adv_dir = os.path.join(main_dir, 'output', 'Persian', 'persian_advs.csv')
persian_intersection_dir = os.path.join(main_dir, 'output', 'Persian', 'persian_inter.csv')

In [3]:
def ADJ_ADV_intersection(adjs: pd.DataFrame, advs: pd.DataFrame) -> pd.DataFrame:
    df1 = adjs[adjs['word'].isin(advs['word'])]
    df2 = advs[advs['word'].isin(adjs['word'])]
    
    df = pd.concat([df1, df2])
    df = df[['word', 'tag', 'sent', 'adj_count', 'adv_count']]
    df = df.sort_values(by= ['word', 'tag'])

    for token in df2.word.unique():
        df.loc[df['word'] == token, 'adj_count'] = len(df1[df1['word'] == token])
        df.loc[df['word'] == token, 'adv_count'] = len(df2[df2['word'] == token])

    df['adj_count'] = df['adj_count'].astype(int)
    df['adv_count'] = df['adv_count'].astype(int)
    
    return df

In [4]:
def create_df(lst: List[Tuple[str, str]], category='adj', threshold=1) -> pd.DataFrame:
    count = category + '_count'

    df = pd.DataFrame(lst, columns =['word', 'sent'])
    df = df.drop_duplicates()
    df.insert(0, count, 0)
    df['tag'] = category

    for token in df['word'].unique():
        df.loc[df['word'] == token, count] = len(df[df['word'] == token])
        
    df = df[df[count] >= threshold].sort_values(by= [count, 'word'], ascending=False)
    
    return df[['word', count, 'tag', 'sent']]

In [5]:
def get_UD_data(dir: str, adj_tags=['ADJ'], adv_tags= ['ADV']) -> (List[Tuple[str, str]], List[Tuple[str, str]]):
     with open(dir) as file:
        lines = file.readlines() 
        ADJs = []
        ADVs = []
        tagset = []
        text = []
        for i, line in enumerate(lines):
            line = line.split()
            if line and i > 0:
                if line[0] == '#' and line[1] == 'text':
                    text = line[3:]
                        
                if line[0].isdigit():
                    token = line[1]
                    tag = line[3]
                    tagset += [tag]
                        
                    if tag in adj_tags:
                        ADJs += [(token, ' '.join(text))]
                    elif tag in adv_tags:
                        ADVs += [(token, ' '.join(text))]
        print(sorted(set(tagset)))
            
        return ADJs, ADVs

In [6]:
persian_adjs, persian_advs = get_UD_data(UD_dir)

['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'VERB', 'X']


In [7]:
persian_adj_df = create_df(persian_adjs, threshold=5)
persian_adv_df = create_df(persian_advs, category='adv', threshold=5)

In [8]:
persian_adj_df[persian_adj_df['word'] == 'پیدا']

,word,adj_count,tag,sent
25,پیدا,638,adj,آتش‌نشانان با استفاده از دستگاه‌های تنفسی بلاف...
310,پیدا,638,adj,در اثر این حادثه رانندهٔ تریلی کشته و سرنشین د...
553,پیدا,638,adj,رفتارهای جدید در میان گروه دلفین‌ها به سرعت ال...
747,پیدا,638,adj,مناظره میان واقع‌گرایی و سازه‌انگاری در حال حا...
776,پیدا,638,adj,با توجه به این که تنها یک هفته تا پایان نیم‌فص...
...,...,...,...,...
31801,پیدا,638,adj,اگر اسلام به معنای واقعی در رسانه‌ها انعکاس پی...
31805,پیدا,638,adj,اختلال در درک «روشنی صدا» عموماً در اثر صدمه ب...
31807,پیدا,638,adj,این خبر تا امروز تقریباً در میان تمام رسانه‌ها...
31808,پیدا,638,adj,تظاهرات در مسکو پایان پیدا کرد.


In [9]:
persian_adj_df[persian_adj_df['word'] == 'کمی']

,word,adj_count,tag,sent
655,کمی,21,adj,این ارزش زیادی دارد که با چنین اعتبار کمی، چنی...
1793,کمی,21,adj,انتشار دوهفته‌نامه چیز کمی نیست.
1922,کمی,21,adj,در آخر خواهیددید آنچه که فکر نمی‌کردید بتوانید...
2278,کمی,21,adj,در طول دو هفته سلول‌های جدید یک قلب ضربان‌دار ...
3370,کمی,21,adj,کارگردان در جلسهٔ نمایش فیلم گله کرده بود از ا...
5852,کمی,21,adj,اهالی یونان باستان درصد بسیار کمی گوشت قرمز در...
7622,کمی,21,adj,تعداد کمی از این قبیل افراد را می‌توان یافت که...
8045,کمی,21,adj,وقت کمی در طول روز برای درس خواندن اختصاص داده...
9755,کمی,21,adj,اگر در هنگام شستشو اتفاقاً مقدار کمی از دهان‌ش...
14347,کمی,21,adj,برای مایع نمودن عسلی که مقدار کمی شکرک زده است...


In [10]:
persian_intersection = ADJ_ADV_intersection(persian_adj_df, persian_adv_df)

In [11]:
persian_intersection

,word,tag,sent,adj_count,adv_count
3603,اندکی,adj,جوانان خام از هر سو به تمنای آن عروس گرد آمدند...,11,9
3780,اندکی,adj,از سال 1970 تا کنون تغییرات اندکی به قانون اسا...,11,9
7086,اندکی,adj,اگر اندکی صبور و شکیبا باشید شادی به خانه شما ...,11,9
8418,اندکی,adj,تغیرات اندکی در زندگی و رفتارت اعمال کن.,11,9
9244,اندکی,adj,به نور ضعیف آتشی می‌مانست که از دور، لحظات اند...,11,9
...,...,...,...,...,...
6913,کمی,adv,در گذشته هر گاه اهل سنت ایران کمی از سوی کشورش...,21,76
6932,کمی,adv,به نظر من کشورهای عضو شورای امنیت، تحریم سوم ر...,21,76
7027,کمی,adv,اگر کمی آشفته به نظر بیایید، مردم شما را فردی ...,21,76
7080,کمی,adv,به نظر من شما در مورد امام خمینی کمی بی‌مهری ک...,21,76


In [12]:
persian_adj_df.to_csv(persian_adj_dir)
persian_adv_df.to_csv(persian_adv_dir)

In [13]:
persian_intersection.to_csv(persian_intersection_dir)